<a href="https://colab.research.google.com/github/amzad-786githumb/AI_and_ML_by-Microsoft/blob/main/34_Implementing_the_troubleshooting_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2> Tasks:</h2>

*  Identify anomalies in system logs.

*  Implement root cause analysis to identify the source of issues.

*  Create solution recommendations based on detected problems.

<h3>Step 1: Set up the environment</h3>

In [2]:
pip install pandas scikit-learn logging

  Using cached logging-0.4.9.6.tar.gz (96 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


<h3>Step 2: Load and explore the dataset</h3>

In [3]:
import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Generate synthetic data
n_samples = 1000
data = {
    'timestamp': pd.date_range(start='2024-01-01', periods=n_samples, freq='h'),
    'cpu_usage': np.random.normal(50, 10, n_samples),       # CPU usage in percentage
    'memory_usage': np.random.normal(60, 15, n_samples),    # Memory usage in percentage
    'network_latency': np.random.normal(100, 20, n_samples), # Network latency in ms
    'disk_io': np.random.normal(75, 10, n_samples),         # Disk I/O in MB/s
    'error_rate': np.random.choice([0, 1], n_samples, p=[0.95, 0.05])  # 5% error rate
}

# Create DataFrame
df = pd.DataFrame(data)

# Display the first few rows of the dataset
print(df.head())
print(df.info())

            timestamp  cpu_usage  memory_usage  network_latency    disk_io  \
0 2024-01-01 00:00:00  54.967142     80.990332        86.496435  55.921924   
1 2024-01-01 01:00:00  48.617357     73.869505        97.109627  66.396150   
2 2024-01-01 02:00:00  56.476885     60.894456        84.151602  70.863945   
3 2024-01-01 03:00:00  65.230299     50.295948        93.840769  93.876877   
4 2024-01-01 04:00:00  47.658466     70.473350        62.127707  80.565531   

   error_rate  
0           0  
1           0  
2           1  
3           0  
4           0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   timestamp        1000 non-null   datetime64[ns]
 1   cpu_usage        1000 non-null   float64       
 2   memory_usage     1000 non-null   float64       
 3   network_latency  1000 non-null   float64       
 4   disk_io

<h3>Step 3: Implement issue detection</h3>

In [4]:
from sklearn.ensemble import IsolationForest

# Implement anomaly detection using Isolation Forest
def detect_anomalies(data):
    model = IsolationForest(contamination=0.05, random_state=42)
    model.fit(data)
    anomalies = model.predict(data)
    return anomalies

# Detect anomalies in the dataset
numeric_data = df.select_dtypes(include=[float, int]) # Only numeric columns
df['anomaly'] = detect_anomalies(numeric_data)

print(df['anomaly'].value_counts()) # -1 denotes an anomaly

anomaly
 1    950
-1     50
Name: count, dtype: int64


<h3>Step 4: Determine which specific value or values made the log anomalous<h3>

In [5]:
from scipy.stats import zscore

# Calculate z-scores to identify anomalous values per column in anomalous rows
z_scores = numeric_data.apply(zscore)

# Function to identify anomalous columns for each row
def find_anomalous_columns(row, threshold=3):
    return [col for col in numeric_data.columns if abs(z_scores.loc[row.name, col]) > threshold]

# Apply the function to each anomalous row
df['anomalous_columns'] = df.apply(lambda row: find_anomalous_columns(row) if row['anomaly'] == -1 else [], axis=1)

# Display rows with anomalies and their anomalous columns
print(df[df['anomaly'] == -1][['timestamp', 'anomaly', 'anomalous_columns']])

              timestamp  anomaly              anomalous_columns
37  2024-01-02 13:00:00       -1                   [error_rate]
38  2024-01-02 14:00:00       -1                   [error_rate]
62  2024-01-03 14:00:00       -1                   [error_rate]
132 2024-01-06 12:00:00       -1                   [error_rate]
179 2024-01-08 11:00:00       -1                   [error_rate]
192 2024-01-09 00:00:00       -1                   [error_rate]
208 2024-01-09 16:00:00       -1                   [error_rate]
241 2024-01-11 01:00:00       -1                   [error_rate]
245 2024-01-11 05:00:00       -1                   [error_rate]
251 2024-01-11 11:00:00       -1                   [error_rate]
262 2024-01-11 22:00:00       -1        [cpu_usage, error_rate]
272 2024-01-12 08:00:00       -1                   [error_rate]
285 2024-01-12 21:00:00       -1                   [error_rate]
315 2024-01-14 03:00:00       -1                   [error_rate]
329 2024-01-14 17:00:00       -1        

<h3>Step 5: Perform root cause analysis</h3>

In [7]:
from sklearn.tree import DecisionTreeClassifier

# Train a decision tree for root cause analysis
def root_cause_analysis(X_train, y_train, X_test):
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    return predictions

# Example root cause analysis (assuming data is preprocessed)
X_train = df.drop(['anomaly', 'timestamp', 'anomalous_columns'], axis=1)
y_train = df['anomaly']
predicted_causes = root_cause_analysis(X_train, y_train, X_train)

# Display predicted causes
print("Predicted causes of anomalies:")
print(pd.Series(predicted_causes).value_counts())

Predicted causes of anomalies:
 1    950
-1     50
Name: count, dtype: int64


<h3>Step 6: Recommend a solution</h3>

In [8]:
# Example solution recommendation based on root cause
def recommend_solution(root_cause):
    solutions = {
        "network_error": "Restart the network service.",
        "database_issue": "Check the database connection and restart the service.",
        "high_cpu_usage": "Optimize running processes or allocate more resources."
    }
    return solutions.get(root_cause, "No recommendation available.")

# Recommend a solution based on a detected root cause
solution = recommend_solution("network_error")
print(f"Recommended solution: {solution}")

Recommended solution: Restart the network service.


<h3>Step 7: Test the troubleshooting agent</h3>

In [10]:
# Simulate a network error by altering the dataset
df.loc[0, 'network_latency'] = 1000  # Simulating high network latency

# Run the troubleshooting agent
numeric_data_test = df.select_dtypes(include=[float, int]) # Select only numeric columns
anomalies = detect_anomalies(numeric_data_test)

# Prepare data for root cause analysis - drop non-feature columns
X_test = numeric_data_test.drop(['anomaly'], axis=1, errors='ignore') # Drop 'anomaly' if it exists in numeric_data_test
predicted_causes = root_cause_analysis(X_train, y_train, X_test)

# Find the index of the modified row (row 0) in the test data
modified_row_index_in_test = X_test.index.get_loc(0)

# Get the predicted cause for the modified row
predicted_cause_for_modified_row = predicted_causes[modified_row_index_in_test]


solution = recommend_solution("network_error") # Assuming network_error is the expected root cause for the simulated anomaly
print(f"Detected issue for row 0: {predicted_cause_for_modified_row}")
print(f"Recommended solution: {solution}")

Detected issue for row 0: 1
Recommended solution: Restart the network service.
